In [2]:
# Get main pointers
import pyMeow as pm
import pymem
import re

def search_data_for_pattern(data: bytes, raw_pattern: str):
    return re.search(
        convert_pattern_to_regex(raw_pattern),
        data,
        re.MULTILINE | re.DOTALL
    )

def convert_pattern_to_regex(pattern: str) -> bytes:

    split_bytes = pattern.split(' ')
    re_pat = bytearray()
    for byte in split_bytes:
        if '?' in byte:
            re_pat.extend(b'.')
        else:
            re_pat.extend(re.escape(bytes.fromhex(byte)))
    return bytes(re_pat)



UWORLDPATTERN =     "48 8B 05 ? ? ? ? 48 8B 88 ? ? ? ? 48 85 C9 74 06 48 8B 49 70"
GOBJECTPATTERN =   "89 0D ? ? ? ? 48 8B DF 48 89 5C 24"
GNAMEPATTERN =      "48 8B 1D ? ? ? ? 48 85 DB 75 ? B9 08 04 00 00"



pid = pm.get_process_id('SoTGame.exe')

process = pymem.Pymem()
process.open_process_from_id(pid)

bulk_scan = process.read_bytes(process.base_address, 100000000)

u_world_base = search_data_for_pattern(bulk_scan, UWORLDPATTERN)
g_object_base = search_data_for_pattern(bulk_scan, GOBJECTPATTERN)
g_name_base = search_data_for_pattern(bulk_scan, GNAMEPATTERN)

print(process.base_address)
print(pid)
print(u_world_base, "\n",g_object_base, "\n", g_name_base)

140694958374912
9852
<re.Match object; span=(8356321, 8356344), match=b'H\x8b\x05\x08\xf2\xe8\x07H\x8b\x88\xc0\x01\x00\x> 
 <re.Match object; span=(25867700, 25867713), match=b'\x89\r&\x8a\xd1\x06H\x8b\xdfH\x89\\$'> 
 <re.Match object; span=(24854906, 24854923), match=b'H\x8b\x1d/\xb8\xe0\x06H\x85\xdbu=\xb9\x08\x04\x0>


In [ ]:
# Addon, gold value on ship
1. Starting from "World" instance, read "OwningGameInstance" as "AthenaGameInstance"
2. From "AthenaGameInstance" go to "AthenaGameContext"
3. From "AthenaGameContext" go to "GameServiceCoordinator" struct
4. From "GameServiceCoordinator" struct, read "Services" array at position 4 to get "RewardService"
5. From "RewardService" read "RewardDefinitionAssets" array
6. For each "RewardDefinitionAsset" instance, read "RewardDefinitions" array
7. For each "RewardDefinition", read "Reward" array at position 0 to get "Reward" struct
8. From "Reward" struct, you get "MinGold" and "MaxGold"

In [ ]:
# detect ship type
maxPlayersOnShip :
	case 2:
		return "Sloop";
		break;
	case 3:
		return "Brigantine";
		break;
	case 4:
		return "Galleon";
		break;
	default:

In [ ]:
# Object type
vector<string> player_objects = { "BP_PlayerPirate_C" };
vector<string> chest_objects = { "BP_AmmoChest_C" };


enum OBJECT_TYPE
{
	PLAYER = 0,
	SHIP = 1,
	CHEST = 2,
	SKULL = 3,
};

enum OBJECT_RARITY
{
	COMMON = 0,
	RARE = 1,
	EPIC = 2,
	MYTHICAL = 3,
	LEGENDARY = 4,
	MERCHANT = 5,
	LORE_BOOK = 6,
	QUEST_ITEM = 7,
	OTHER = 8,


In [ ]:
# all crew data
gamestate = 0x58
AthenaGameState = process.read_longlong(oworld + gamestate)

if AthenaGameState == 0x0 :
    print("not in game")
else :
    CrewService = process.read_longlong(AthenaGameState + 0x608)
    Crews = process.read_longlong(CrewService + 0x4A0)
    print("crews ptr : ", hex(Crews))

crews_list = []
for i in range(0,20):
    try :
        crew_ptr = hex(process.read_longlong(Crews + 0x20+i*0x98))

        if crew_ptr[2]+crew_ptr[3] == hex(oplayer)[2] + hex(oplayer)[3] :                # Supprimmable quand on aura les noms
            crews_list.append(crew_ptr)
        else :
            break
    except :
        print("error reading crew pointer")
        None


print("all crew pointers : ", crews_list)

for crew in crews_list :
    crew_list = []
    for player in range(0,3) :
        try :
            player_ptr = hex(process.read_longlong(int(crew, 16)+player*0x8))
            print(process.read_string(int(player_ptr,16)+0x710,255))
            if player_ptr[2]+player_ptr[3] == hex(oplayer)[2] + hex(oplayer)[3] :       # Supprimmable quand on aura les noms
                crew_list.append(player_ptr)
        except :
            None
    print(crew_list)


In [ ]:
# player controller to manager
playercontroller = process.read_longlong(oplayer+0x30)
manager = process.read_longlong(oplayer+0x458)

hex(manager)

In [ ]:
# Search string in files
import os
import json
search = "structCrews"
for filename in os.listdir("JSON-SDK"):
    f = os.path.join("JSON-SDK", filename)
    if os.path.isfile(f):
        with open(f) as json_file:
            data = json.load(json_file)
            if search in data :
                print(f)

In [ ]:
# chercher pseudo nul
for i in range(0,1500):
    try :
        print(process.read_string(oplayer+i*0x04,255))
    except :
        a = 1

In [ ]:
# Advanced map
import pyMeow as pm
import time
from fonctions import *


map_full, width, height, show_menu, color_black, color_white, color_red, color_green, color_blue = fct.initialisation()

map_pointer = pm.load_texture("player_pointer.png")

offset_menu_x = width/4
offset_menu_y = height/4
menu_thickness = 150
test = pm.load_texture("test.png")

reseted = False
while pm.overlay_loop():
    pm.begin_drawing()
    show_menu = fct.menu_status(show_menu)  # change variable affichage menue

    player_x, player_y = fct.grab_data()    # Simulation XY


    # work texture
    pm.draw_rectangle_lines(offset_menu_x, offset_menu_y, width/2, height/2, color_blue, 1)    # main menu
    pm.draw_rectangle_lines(offset_menu_x, offset_menu_y-menu_thickness, width/2, menu_thickness, color_red, 1)    # letter menu
    pm.draw_rectangle_lines(offset_menu_x-menu_thickness, offset_menu_y, menu_thickness, height/2, color_green, 1)    # number menu


    # IHM
    pm.gui_button(offset_menu_x+width/2+50, offset_menu_y, 50, 50, "+")
    pm.gui_button(offset_menu_x+width/2+50, offset_menu_y+100, 50, 50, "-")

    pm.gui_button(offset_menu_x+width/2+50, offset_menu_y+250, 50, 50, "up")
    pm.gui_button(offset_menu_x+width/2+50, offset_menu_y+300, 50, 50, "down")
    pm.gui_button(offset_menu_x+width/2+50-50, offset_menu_y+300, 50, 50, "left")
    pm.gui_button(offset_menu_x+width/2+50+50, offset_menu_y+300, 50, 50, "right")


    if show_menu == True :
        reseted = False
        pm.draw_texture(map_full, offset_menu_x+1, offset_menu_y+1, color_white, 0, 0.3)
    else :
        if reseted == False :
            reseted = True
            print("reset affichage map")


    pm.end_drawing()



In [ ]:
# button with max value
        if pm.gui_button(0, 0, 100, 50, "Increase speed"):
            if speed != max_speed:
                speed += 5